In [ ]:
import geopandas as gpd
from owslib.wfs import WebFeatureService
from requests import Request

Det er den her der virker


In [61]:
base_url = "https://geoserver.plandata.dk/geoserver/wfs"
wfs = WebFeatureService(url=base_url, version='2.0.0')

headers = {
    # ... your headers if any ...
}
for layer in wfs.contents:
    
    startIndex = 0
    count = 1000
    all_content = ''
    layer_file_name =layer.replace(':', '_')
    print(layer)
    while True:
        print("run")
        params = {
            "service": "WFS",
            "version": "2.0.0",
            "request": "GetFeature",
            "typeNames": "lpd:lpd_bp_Stationer",  # replace with the correct typeName
            'startIndex': startIndex,
            'count': count
        }
        
        response = requests.get(base_url, headers=headers, params=params)
        
        content = response.content.decode('utf-8')
        
        # If it's the first iteration, store everything up to the first <wfs:member> tag
        if startIndex == 0:
            all_content += content.split('<wfs:member>')[0]

        # Extract and store all <wfs:member> sections
        members = content.split('<wfs:member>')[1:]
        for member in members:
            all_content += '<wfs:member>' + member.split('</wfs:member>')[0] + '</wfs:member>'
        
        # Check if there's more to fetch
        if content.count('<wfs:member>') < count:
            break

        startIndex += count

    # Add the closing tag
    all_content += "</wfs:FeatureCollection>"

    # Save to a GML file
    with open('./data/'+ layer_file_name +'.gml', 'w') as file:
        file.write(all_content)


run


In [ ]:
print(all_content)

In [ ]:
print(root)

In [ ]:
wfs = WebFeatureService(url='https://geoserver.plandata.dk/geoserver/wfs', version='2.0.0')
base_request_str = 'https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames='

# Create an empty GeoPackage file or use an existing one
output_gpkg = './data/plandata.gpkg'

for layer in wfs.contents:
    print(layer)
    url = base_request_str + layer

    q = Request('GET', url).prepare().url
    try:
        df = gpd.read_file(q, format='GML')
        df.crs = 'EPSG:25832'
        layer_file_bame =layer.replace(':', '_')
        #df.to_file(layer_file_bame+'.shp')
        gdf.to_file(output_gpkg, layer=layer, driver="GPKG", mode='a' if os.path.exists(output_gpkg) else 'w')
    except Exception as e:
        print(f"An error occurred: {e}")
        continue  # This is optional as the loop will naturally go to the next iteration
    



this can be expanded to 

    First request that starts from the beginning

http://wms.pcn.minambiente.it/ogc?map=/ms_ogc/wfs/Edifici.map&service=WFS&version=1.1.0&request=GetFeature&typename=ED.EDIFICATO.CAPOLUOGHI.&maxfeatures=1000

Save response into file

    Second request

http://wms.pcn.minambiente.it/ogc?map=/ms_ogc/wfs/Edifici.map&service=WFS&version=1.1.0&request=GetFeature&typename=ED.EDIFICATO.CAPOLUOGHI.&MaxFeatures=1000&StartIndex=1000

In [ ]:
def download_wfs_layers_to_geopackage(url, output_filepath):
    # Connect to the WFS service
    wfs = WebFeatureService(url=url)
    
    # Check WFS version and set typename parameter accordingly
    typename_param = 'typename'
    if wfs.version == '2.0.0':
        typename_param = 'typenames'

    # For each layer, download its data
    for layer in wfs.contents:
        print(layer)
        # Check if layer name contains namespace (like 'namespace:layername')
        if ':' not in layer:
            layer = f"{wfs.provider.name}:{layer}"
        
        params = {
            typename_param: layer,
            'outputFormat': 'application/json'
        }
        
        try:
            response = wfs.getfeature(**params)
            data = response.read()
            gdf = gpd.read_file(data)
            
            # Append the data to the GeoPackage
            mode = 'a' if 'tables' in gpd.io.file.fiona.drvsupport.supported_drivers['GPKG'] and layer in gpd.io.file.fiona.drvsupport.supported_drivers['GPKG']['tables'] else 'w'
            gdf.to_file(output_filepath, layer=layer, driver="GPKG", mode=mode)
            print(f"Saved {layer} to {output_filepath}")
        except Exception as e:
            print(f"Error fetching data for layer {layer}: {e}")


if __name__ == '__main__':
    wfs_url = 'https://geoserver.plandata.dk/geoserver/wfs?servicename=wfs&request=getcapabilities&service=wfs'
    output_filepath = './data/plandata.gpkg'  # e.g., './data.gpkg'
    download_wfs_layers_to_geopackage(wfs_url, output_filepath)


In [ ]:
from owslib.wfs import WebFeatureService
wfs = WebFeatureService(url='https://geoserver.plandata.dk/geoserver/wfs', version='2.0.0')


In [ ]:
[operation.name for operation in wfs11.operations]

In [ ]:
list(wfs11.contents)

In [ ]:
from requests import Request
#url = 'http://wms.pcn.minambiente.it/ogc?map=/ms_ogc/wfs/Edifici.map&service=wfs&request=getFeature&version=1.0.0&typename=ED.EDIFICATO.CAPOLUOGHI.'
url = 'https://geoserver.plandata.dk/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=lpd:lpd_bp_Stationer'


q = Request('GET', url).prepare().url
df = gpd.read_file(q, format='GML')
df.crs = 'EPSG:25832'

df.to_file('output.shp')

In [ ]:
import geopandas as gpd
from owslib.wfs import WebFeatureService
import os

# Define the WFS endpoint
url = 'https://geoserver.plandata.dk/geoserver/wfs'

# Connect to the WFS service
wfs = WebFeatureService(url)

# List feature types (layers) available in the service
layers = list(wfs.contents.keys())

# Create an empty GeoPackage file or use an existing one
output_gpkg = 'output.gpkg'

for layer in layers:
    # Build the request URL for each layer
    print(layer)
    layer_url = f"{url}?service=WFS&version=2.0.0&request=GetFeature&typeNames={layer}"
    # Read the layer into a GeoDataFrame
    gdf = gpd.read_file(layer_url)
    
    # Set the Coordinate Reference System (CRS)
    gdf.crs = 'EPSG:25832'
    
    # Save the GeoDataFrame to the GeoPackage
    gdf.to_file(output_gpkg, layer=layer, driver="GPKG", mode='a' if os.path.exists(output_gpkg) else 'w')

print(f"All layers saved to {output_gpkg}")

In [ ]:
[storedquery.id for storedquery in wfs11.storedqueries]

In [ ]:
[parameter.name for parameter in wfs11.storedqueries[0].parameters]

In [ ]:
response = wfs11.getfeature(storedQueryID='urn:ogc:def:query:OGC-WFS::GetFeatureById', storedQueryParams={'ID':'gmd_ex.1'})

[operation.name for operation in wfs11.operations]